In [1]:
#import bibliotek
from kafka import KafkaConsumer
import ast
import time
from csv import DictWriter
import uuid

In [2]:
topic = 'quickstart-events'
field_names = ['id', 'name', 'age', 'job', 'marital', 'education', 'default', 'housing', 'loan']
consumer = KafkaConsumer(topic, bootstrap_servers=['127.0.0.1:9092'])

In [3]:
#funkcja do przetworzania slownika w wiersze 

def append_dict_as_row(file_name, dict_of_elem, field_names):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        dict_writer = DictWriter(write_obj, fieldnames=field_names)
        # Add dictionary as wor in the csv
        dict_writer.writerow(dict_of_elem)

In [4]:
for message in consumer:
    print(message.value)
    dict_str = message.value.decode("UTF-8")
    temp = ast.literal_eval(dict_str)
    temp[''] = model.predict()
    temp['id'] = uuid.uuid4()
    append_dict_as_row('clients.csv', temp, field_names)
    time.sleep(4)

b'{"name": "Karol Duda", "age": "30", "job": "entrepreneur", "marital": "married", "education": "high.school", "default": "no", "housing": "no", "loan": "no"}'
b'{"name": "Peter Green", "age": "45", "job": "entrepreneur", "marital": "divorced", "education": "university.degree", "default": "no", "housing": "yes", "loan": "no"}'
b'{"name": "Lence Doe-Marrow", "age": "28", "job": "management", "marital": "divorced", "education": "university.degree", "default": "yes", "housing": "yes", "loan": "yes"}'
b'{"name": "Test_user", "age": "23", "job": "management", "marital": "divorced", "education": "university.degree", "default": "no", "housing": "no", "loan": "no"}'
b'{"name": "John Doe", "age": "28", "job": "management", "marital": "single", "education": "university.degree", "default": "yes", "housing": "yes", "loan": "yes"}'
b'{"name": "Test_user2", "age": "55", "job": "management", "marital": "divorced", "education": "university.degree", "default": "no", "housing": "no", "loan": "no"}'
b'nf

ValueError: malformed node or string: <_ast.Name object at 0x00000214BFF4F3D0>